In [ ]:
# ============================================================
# OMEGA INFINITY v4 - META NIVELES RECURSIVOS
# ============================================================
# f(m) = m aplicado N veces: f(f(f(...f(m)...)))
# ============================================================

import numpy as np
from collections import defaultdict
import struct
import zlib

class MetaMarkovCompressor:
    """Compresor base de Markov adaptativo"""

    def __init__(self, order=3):
        self.order = order

    def compress_to_ranks(self, text):
        """Comprime texto a lista de ranks + metadata"""
        if len(text) <= self.order:
            return {
                'prefix': text,
                'ranks': [],
                'max_values': [],
                'alphabet': list(set(text)) if text else []
            }

        transitions = defaultdict(lambda: defaultdict(int))
        alphabet = sorted(set(text))
        char_to_idx = {c: i for i, c in enumerate(alphabet)}

        prefix = text[:self.order]
        ranks = []
        max_values = []

        for i in range(self.order, len(text)):
            context = text[i-self.order:i]
            current_char = text[i]

            if context in transitions:
                seen_chars = sorted(
                    transitions[context].items(),
                    key=lambda x: (-x[1], x[0])
                )
                seen_list = [c for c, _ in seen_chars]

                if current_char in seen_list:
                    rank = seen_list.index(current_char)
                    max_val = len(seen_list) + 1
                else:
                    rank = len(seen_list)
                    max_val = len(seen_list) + 1

                ranks.append(rank)
                max_values.append(max_val)

                if current_char not in seen_list:
                    unseen = [c for c in alphabet if c not in seen_list]
                    unseen_idx = unseen.index(current_char)
                    ranks.append(unseen_idx)
                    max_values.append(len(unseen))
            else:
                rank = char_to_idx[current_char]
                ranks.append(rank)
                max_values.append(len(alphabet))

            transitions[context][current_char] += 1

        return {
            'prefix': prefix,
            'ranks': ranks,
            'max_values': max_values,
            'alphabet': alphabet,
            'original_len': len(text)
        }

    def decompress_from_ranks(self, data):
        """Reconstruye texto desde ranks"""
        prefix = data['prefix']
        ranks = data['ranks']
        max_values = data['max_values']
        alphabet = data['alphabet']
        original_len = data['original_len']

        if not ranks:
            return prefix

        result = list(prefix)
        transitions = defaultdict(lambda: defaultdict(int))
        char_to_idx = {c: i for i, c in enumerate(alphabet)}

        rank_idx = 0

        while len(result) < original_len and rank_idx < len(ranks):
            context = ''.join(result[-self.order:])

            if context in transitions:
                seen_chars = sorted(
                    transitions[context].items(),
                    key=lambda x: (-x[1], x[0])
                )
                seen_list = [c for c, _ in seen_chars]

                rank = ranks[rank_idx]
                rank_idx += 1

                if rank < len(seen_list):
                    char = seen_list[rank]
                else:
                    unseen = [c for c in alphabet if c not in seen_list]
                    unseen_idx = ranks[rank_idx]
                    rank_idx += 1
                    char = unseen[unseen_idx]
            else:
                rank = ranks[rank_idx]
                rank_idx += 1
                char = alphabet[rank]

            result.append(char)
            transitions[context][char] += 1

        return ''.join(result)


class OmegaInfinityRecursive:
    """
    Compresión Meta-Recursiva: f(f(f(m)))
    Aplica Markov sobre los ranks, luego sobre esos ranks, etc.
    """

    def __init__(self, base_order=3, meta_levels=3):
        self.base_order = base_order
        self.meta_levels = meta_levels

    def compress(self, text):
        print(f"{'='*60}")
        print(f"🌌 OMEGA INFINITY v4 - META RECURSIVO")
        print(f"{'='*60}")
        print(f"📝 Texto original: {len(text)} caracteres")
        print(f"📊 Orden base: {self.base_order}")
        print(f"🔄 Meta-niveles: {self.meta_levels}")
        print(f"{'='*60}\n")

        # Guardar metadatos de cada nivel
        levels_data = []
        current_text = text

        for level in range(self.meta_levels):
            print(f"⚙️  Nivel {level + 1}:")
            print(f"   📥 Input: {len(current_text)} símbolos")

            # Ajustar orden según el nivel
            order = max(1, self.base_order - level)
            compressor = MetaMarkovCompressor(order=order)

            # Comprimir a ranks
            data = compressor.compress_to_ranks(current_text)

            print(f"   📦 Ranks generados: {len(data['ranks'])}")
            print(f"   📉 Max rank: {max(data['ranks']) if data['ranks'] else 0}")
            print(f"   🔤 Alfabeto: {len(data['alphabet'])} símbolos")

            # Guardar metadata de este nivel
            level_meta = {
                'order': order,
                'prefix': data['prefix'],
                'alphabet': data['alphabet'],
                'max_values': data['max_values'],
                'original_len': data['original_len']
            }
            levels_data.append(level_meta)

            # Convertir ranks a string para siguiente nivel
            # Usar caracteres para representar ranks
            if data['ranks']:
                # Mapear ranks a caracteres Unicode
                max_rank = max(data['ranks'])
                if max_rank < 256:
                    # Usar bytes directamente
                    current_text = ''.join(chr(r) for r in data['ranks'])
                else:
                    # Usar representación con separador
                    current_text = ','.join(map(str, data['ranks']))
            else:
                current_text = ""
                break

            print(f"   📤 Output: {len(current_text)} símbolos")

            # Si ya es muy pequeño, parar
            if len(current_text) < 10:
                print(f"   ⏹️  Suficientemente pequeño, parando.")
                break

            print()

        # El último nivel: guardar los ranks finales directamente
        final_ranks = [ord(c) for c in current_text] if current_text else []

        # Empaquetar todo
        package = self._pack_all(levels_data, final_ranks)
        compressed = zlib.compress(package, level=9)

        # Estadísticas
        original_size = len(text.encode('utf-8'))
        compressed_size = len(compressed)
        ratio = (1 - compressed_size / original_size) * 100

        print(f"\n{'='*60}")
        print(f"📊 RESULTADOS FINALES")
        print(f"{'='*60}")
        print(f"📁 Original: {original_size} bytes")
        print(f"📦 Comprimido: {compressed_size} bytes")
        print(f"📉 Ratio: {ratio:.2f}%")
        print(f"🔢 Factor: {original_size/compressed_size:.2f}x")
        print(f"🔄 Niveles usados: {len(levels_data)}")
        print(f"{'='*60}")

        return compressed, levels_data, final_ranks

    def _pack_all(self, levels_data, final_ranks):
        """Empaqueta todos los niveles"""
        package = bytearray()

        # Número de niveles
        package.append(len(levels_data))

        # Cada nivel
        for level in levels_data:
            # Order
            package.append(level['order'])

            # Original length
            package.extend(struct.pack('I', level['original_len']))

            # Prefix
            prefix_bytes = level['prefix'].encode('utf-8')
            package.extend(struct.pack('H', len(prefix_bytes)))
            package.extend(prefix_bytes)

            # Alphabet
            alphabet_str = ''.join(level['alphabet'])
            alphabet_bytes = alphabet_str.encode('utf-8')
            package.extend(struct.pack('H', len(alphabet_bytes)))
            package.extend(alphabet_bytes)

            # Max values
            package.extend(struct.pack('I', len(level['max_values'])))
            for mv in level['max_values']:
                package.extend(struct.pack('H', min(mv, 65535)))

        # Final ranks
        package.extend(struct.pack('I', len(final_ranks)))
        for r in final_ranks:
            package.extend(struct.pack('H', r))

        return bytes(package)

    def decompress(self, compressed):
        """Descomprime recursivamente"""
        print(f"\n{'='*60}")
        print(f"🔓 DESCOMPRESIÓN META-RECURSIVA")
        print(f"{'='*60}")

        data = zlib.decompress(compressed)
        pos = 0

        # Número de niveles
        num_levels = data[pos]; pos += 1
        print(f"🔄 Niveles a procesar: {num_levels}")

        # Leer cada nivel
        levels_data = []
        for i in range(num_levels):
            level = {}

            level['order'] = data[pos]; pos += 1
            level['original_len'] = struct.unpack('I', data[pos:pos+4])[0]; pos += 4

            prefix_len = struct.unpack('H', data[pos:pos+2])[0]; pos += 2
            level['prefix'] = data[pos:pos+prefix_len].decode('utf-8'); pos += prefix_len

            alphabet_len = struct.unpack('H', data[pos:pos+2])[0]; pos += 2
            level['alphabet'] = list(data[pos:pos+alphabet_len].decode('utf-8')); pos += alphabet_len

            num_max_values = struct.unpack('I', data[pos:pos+4])[0]; pos += 4
            level['max_values'] = []
            for _ in range(num_max_values):
                mv = struct.unpack('H', data[pos:pos+2])[0]; pos += 2
                level['max_values'].append(mv)

            levels_data.append(level)

        # Final ranks
        num_final = struct.unpack('I', data[pos:pos+4])[0]; pos += 4
        final_ranks = []
        for _ in range(num_final):
            r = struct.unpack('H', data[pos:pos+2])[0]; pos += 2
            final_ranks.append(r)

        # Reconstruir desde el último nivel hacia el primero
        current_ranks = final_ranks

        for i in range(num_levels - 1, -1, -1):
            level = levels_data[i]
            print(f"   ⬆️  Nivel {i + 1}: {len(current_ranks)} ranks → ", end="")

            # Crear datos para descomprimir
            decomp_data = {
                'prefix': level['prefix'],
                'ranks': current_ranks,
                'max_values': level['max_values'],
                'alphabet': level['alphabet'],
                'original_len': level['original_len']
            }

            compressor = MetaMarkovCompressor(order=level['order'])
            text = compressor.decompress_from_ranks(decomp_data)

            print(f"{len(text)} símbolos")

            # Para el siguiente nivel, convertir texto a ranks
            if i > 0:
                current_ranks = [ord(c) for c in text]
            else:
                result = text

        print(f"\n✅ Texto recuperado: {len(result)} caracteres")
        print(f"{'='*60}")

        return result


# ============================================================
# PRUEBAS
# ============================================================

def test_recursive():
    textos = {
        "repetitivo": "abracadabra " * 100,
        "natural": """La inteligencia artificial es una rama de la informática
        que busca crear sistemas capaces de realizar tareas.""" * 10,
        "código": "def f(n):\n    if n <= 1:\n        return n\n    return f(n-1) + f(n-2)\n" * 20,
        "binario": "01001011" * 200,
        "adn": "ATCGATCGATCGATCG" * 100,
    }

    print("\n" + "🌌"*30)
    print("  OMEGA INFINITY v4 - META RECURSIVO")
    print("  f(f(f(m))) - Compresión Multi-Nivel")
    print("🌌"*30 + "\n")

    resultados = []

    for nombre, texto in textos.items():
        print(f"\n{'#'*60}")
        print(f"# {nombre.upper()}")
        print(f"{'#'*60}")

        comp = OmegaInfinityRecursive(base_order=4, meta_levels=3)

        try:
            compressed, _, _ = comp.compress(texto)
            recovered = comp.decompress(compressed)

            match = texto == recovered
            print(f"\n🔍 Verificación: {'✅ CORRECTO' if match else '❌ ERROR'}")

            if not match:
                for i in range(min(len(texto), len(recovered))):
                    if texto[i] != recovered[i]:
                        print(f"   Diff en pos {i}: '{texto[i]}' vs '{recovered[i]}'")
                        break

            original_size = len(texto.encode('utf-8'))
            compressed_size = len(compressed)

            resultados.append({
                'nombre': nombre,
                'original': original_size,
                'comprimido': compressed_size,
                'factor': original_size / compressed_size,
                'ratio': (1 - compressed_size/original_size) * 100,
                'match': match
            })

        except Exception as e:
            print(f"❌ Error: {e}")
            import traceback
            traceback.print_exc()

    # Resumen
    print(f"\n\n{'='*70}")
    print("📊 RESUMEN FINAL - META RECURSIVO")
    print(f"{'='*70}")
    print(f"{'Texto':<12} {'Original':>10} {'Comprimido':>12} {'Factor':>8} {'Ratio':>10} {'OK':>5}")
    print(f"{'-'*70}")
    for r in resultados:
        status = "✅" if r['match'] else "❌"
        print(f"{r['nombre']:<12} {r['original']:>10} {r['comprimido']:>12} {r['factor']:>7.1f}x {r['ratio']:>9.1f}% {status:>5}")

    print(f"\n🏆 Mejor factor: {max(r['factor'] for r in resultados):.1f}x")

test_recursive()